In [1]:
import pandas as pd 
import os
import glob


In [2]:
n=8

mdf=pd.read_excel(f'/workspaces/vikrant_dubai/RAW/AD/AD {n}.xlsx')
excel_file = pd.ExcelFile(f'/workspaces/vikrant_dubai/RAW/AD/AD {n}.xlsx')

In [3]:
columns_of_main_df = ['Station Name','Date', 'Time', 'SO2', 'Data Cap. (SO2)', 'SO2 Avg', 'NO2', 'Data Cap. (NO2)', 'NO2 Avg',      'O3', 'Data Cap. (O3)', 'O3 8H Avg.', 'Daily Capture O3', 'O3 Daily Average', 'O3 Daily Average (NS)', 'CO',  'Data Cap. (CO)', 'CO 8H Avg.', 'Daily Capture CO', 'CO Daily Average', 'PM10',    'Data Cap. (PM10)', 'PM10 Avg', 'PM2.5', 'Data Cap. (PM2.5)', 'PM2.5 Avg', 'Lower Ambient Temperature', 'Upper Ambient Temperature', 'Barometric Pressure', 'Relative Humidity', 'Wind Direction', 'Wind Speed', 'H2S', 'Toluene', 'O-Xylene', 'Ethylbenzene', 'MP-Xylene', 'Benzene', 'CH4', 'NMHC', 'THC', 'Noise', 'Solar Radiation','Atm P.']
for s in range(len(excel_file.sheet_names)-4):
    main_df = pd.DataFrame(columns=columns_of_main_df)

    # Read the Excel file and preprocess the data
    mdf = pd.read_excel(f'/workspaces/vikrant_dubai/RAW/AD/AD {n}.xlsx', sheet_name=excel_file.sheet_names[s])
    mdf = mdf.drop([0, 1, 2, 3]).reset_index(drop=True)
    mdf.columns = mdf.iloc[0]
    mdf = mdf[1:].reset_index(drop=True).drop([0, 1]).reset_index(drop=True)

    # Convert 'Date/Time' column to separate 'Date' and 'Time' columns
    mdf['Date/Time'] = pd.to_datetime(mdf['Date/Time'])
    mdf['Date'] = mdf['Date/Time'].dt.date
    mdf['Time'] = mdf['Date/Time'].dt.time

    # Iterate over the series and replace the column names
    new_df_columns = mdf.columns.tolist()
    for col_index in range(1, len(new_df_columns)):
        if 'DATA CAP.' in new_df_columns[col_index]:
            if new_df_columns[col_index-1] == 'DATA COUNT':
                new_df_columns[col_index] = f"DATA CAP. ({new_df_columns[col_index-2]})"
            else:
                new_df_columns[col_index] = f"DATA CAP. ({new_df_columns[col_index-1]})"
        if 'Daily Capture' in new_df_columns[col_index]:
            new_df_columns[col_index] = f"Daily Capture {new_df_columns[col_index-1]}"
            if '8H Avg.' in new_df_columns[col_index-1]:
                new_df_columns[col_index] = new_df_columns[col_index].replace('8H Avg.', '')
    mdf.columns = new_df_columns

    # Reorder columns and drop the original 'Date/Time' column
    mdf = mdf[['Date', 'Time'] + [col for col in mdf.columns if col not in ['Date', 'Time']]]
    mdf = mdf.drop(columns=['Date/Time'])

    # add column station name
    mdf['Station Name'] = excel_file.sheet_names[s]

    # Format the 'Date' column
    mdf['Date'] = pd.to_datetime(mdf['Date']).dt.strftime('%d/%m/%Y')
    break

In [8]:
import difflib

# List of values to find matches for
values = [col.upper() for col in mdf.columns]

# List of column names
l1 = [col.upper() for col in main_df.columns]

# Loop through values and find the closest match
matches = {}
for value in values:
    closest_match = difflib.get_close_matches(value, l1, n=1)
    if closest_match:
        main_df[main_df.columns[main_df.columns.str.upper() == closest_match[0]][0]] = mdf[mdf.columns[mdf.columns.str.upper() == value][0]]
    else:
        main_df[value] = ''


In [9]:
main_df

,Station Name,Date,Time,SO2,Data Cap. (SO2),SO2 Avg,NO2,Data Cap. (NO2),NO2 Avg,O3,...,Ethylbenzene,MP-Xylene,Benzene,CH4,NMHC,THC,Noise,Solar Radiation,Atm P.,DATA COUNT
0,Hamdan Street,01/08/2023,00:00:00,24.38,NaN,18.14,145.61,NaN,44.72,NaN,...,12.75,46.29,11.61,NaN,NaN,NaN,67.53,NaN,NaN,
1,Hamdan Street,01/08/2023,01:00:00,24.58,NaN,18.14,137.1,NaN,44.72,NaN,...,25.07,83.2,14.87,NaN,NaN,NaN,66.44,NaN,NaN,
2,Hamdan Street,01/08/2023,02:00:00,23.32,NaN,18.14,58.18,NaN,44.72,NaN,...,17.74,65.79,10.45,NaN,NaN,NaN,66.55,NaN,NaN,
3,Hamdan Street,01/08/2023,03:00:00,25.77,NaN,18.14,44.26,NaN,44.72,NaN,...,7.95,30.15,5.7,NaN,NaN,NaN,64.37,NaN,NaN,
4,Hamdan Street,01/08/2023,04:00:00,20.16,NaN,18.14,69.45,NaN,44.72,NaN,...,14.12,44.14,5.05,NaN,NaN,NaN,62.89,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,Hamdan Street,31/08/2023,19:00:00,12.92,NaN,7.57,21.83,NaN,22.76,NaN,...,3.6,13.03,2.99,NaN,NaN,NaN,68.7,NaN,NaN,
740,Hamdan Street,31/08/2023,20:00:00,4.17,NaN,7.57,28.22,NaN,22.76,NaN,...,4.36,17.58,3.86,NaN,NaN,NaN,67.77,NaN,NaN,
741,Hamdan Street,31/08/2023,21:00:00,4.36,NaN,7.57,35.91,NaN,22.76,NaN,...,5.46,22.15,4.65,NaN,NaN,NaN,68.9,NaN,NaN,
742,Hamdan Street,31/08/2023,22:00:00,4.32,NaN,7.57,32.09,NaN,22.76,NaN,...,6.94,28.47,5.19,NaN,NaN,NaN,69.05,NaN,NaN,
